In [39]:
import finvizfinance as fz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from finvizfinance.quote import finvizfinance, Statements, Quote
import yfinance as yf


In [40]:
tickers = [
    "EQIX", "DLR",
    "PWR", "ETN", "ABB",
    "VRT", "TT", "JCI",
    "SMCI", "HPE", "DELL",
    "ANET", "AVGO", "CSCO",
    "MRVL", "COHR", "LITE", "INFN",
    "MU", "ON", "STM", "ADI", "TXN",
    "TSM", "INTC", "ASX", "AMKR", "PLAB",
    "SNPS", "CDNS", "ANSS",
    "STX", "WDC",
    "ASML", "AMAT", "KLAC", "LRCX",
    "NVDA",
    "GOOG", "GOOGL", "ADBE"
]

# tickers = ["TSM", "GOOGL", "ADBE"]


In [41]:
import math

# ---------- helpers ----------
def _to_float(x, default=0.0):
    if x is None:
        return default
    s = str(x).replace(',', '').strip()
    if s.endswith('%'):
        s = s[:-1]
    try:
        return float(s)
    except:
        return default

def _pos(x, lo, hi):
    """Higher is better. Linear 0..1 between [lo, hi]."""
    if x <= lo: return 0.0
    if x >= hi: return 1.0
    return (x - lo) / (hi - lo)

def _neg(x, lo, hi):
    """Lower is better. Linear 0..1 between [lo, hi]."""
    if x <= lo: return 1.0
    if x >= hi: return 0.0
    return 1.0 - (x - lo) / (hi - lo)

def _series_recent4(df, row_label):
    """Return list of 4 most-recent values for a given row (columns are most-recent first)."""
    if row_label not in df.index:
        return [0.0, 0.0, 0.0, 0.0]
    row = df.loc[row_label]
    vals = [_to_float(row.iloc[i], 0.0) for i in range(min(4, row.shape[0]))]
    while len(vals) < 4:
        vals.append(vals[-1] if vals else 0.0)
    return vals

def _weighted_recent4(values, weights=(1.0, 0.75, 0.50, 0.25)):
    return sum(values[i]*weights[i] for i in range(4)) / sum(weights)

# ---------- metric extraction from dfs ----------
def extract_from_quarterlies(income_df, cashflow_df, balance_df, fundamentals_dict):
    # Revenue TTM growth (current 4q vs previous 4q)
    rev_row = income_df.loc['Total Revenue'] if 'Total Revenue' in income_df.index else None
    if rev_row is not None and rev_row.shape[0] >= 8:
        rev_now  = sum(_to_float(rev_row.iloc[i])   for i in range(4))      # cols 0..3
        rev_prev = sum(_to_float(rev_row.iloc[i])   for i in range(4, 8))   # cols 4..7
        rev_ttm_growth = ( (rev_now - rev_prev)/rev_prev*100.0 ) if rev_prev > 0 else 0.0
    else:
        # Fallback to fundamentals 'Sales Y/Y TTM'
        rev_ttm_growth = _to_float(fundamentals_dict.get('Sales Y/Y TTM', 0.0))

    # FCF margin: weighted last 4 quarters (linear decay)
    fcf4 = _series_recent4(cashflow_df, 'Free Cash Flow')
    rev4 = _series_recent4(income_df, 'Total Revenue')
    fcf_w = _weighted_recent4(fcf4)
    rev_w = _weighted_recent4(rev4)
    fcf_margin = (fcf_w/rev_w*100.0) if rev_w > 0 else 0.0

    # Safety: Current Ratio from balance sheet (latest column 0)
    curr_ratio = _to_float(balance_df.loc['Current Ratio'].iloc[0]) if 'Current Ratio' in balance_df.index else _to_float(fundamentals_dict.get('Current Ratio', 1.0))

    # Debt/Equity: use fundamentals (simple and available)
    debt_eq = _to_float(fundamentals_dict.get('Debt/Eq', 0.0))

    return rev_ttm_growth, fcf_margin, debt_eq, curr_ratio

# ---------- final score ----------
def buy_score(stock_fundamentals, income_df, cashflow_df, balance_df):
    # From fundamentals (all are % except valuation)
    gm  = _to_float(stock_fundamentals.get('Gross Margin', 0))     # %
    om  = _to_float(stock_fundamentals.get('Oper. Margin', 0))     # %
    roe = _to_float(stock_fundamentals.get('ROE', 0))              # %
    fpe = _to_float(stock_fundamentals.get('Forward P/E', 0))      # number
    peg = _to_float(stock_fundamentals.get('PEG', 0))              # number

    # From dfs (or simple fallbacks)
    rev_g, fcf_margin, debt_eq, curr_ratio = extract_from_quarterlies(
        income_df, cashflow_df, balance_df, stock_fundamentals
    )

    # Subscores (0..1) — keep ranges simple and sane for a 2-year horizon
    growth = _pos(rev_g, 5, 40)  # 5%~40% revenue TTM growth
    profitability = (
        _pos(gm, 40, 70) +
        _pos(om, 15, 45) +
        _pos(roe, 10, 40) +
        _pos(fcf_margin, 5, 35)   # strong cash conversion
    ) / 4.0
    valuation = (
        _neg(fpe, 12, 40) +
        _neg(peg, 1.0, 3.0)
    ) / 2.0
    safety = (
        _neg(debt_eq, 0.0, 1.0) +
        _pos(curr_ratio, 1.0, 3.0)
    ) / 2.0

    score = (
        0.30*growth +
        0.30*profitability +
        0.25*valuation +
        0.15*safety
    ) * 100.0

    return round(score, 1)

def score_label(score):
    if score >= 75: return "BUY (High Conviction)"
    if score >= 60: return "ACCUMULATE / HOLD"
    return "AVOID / WATCHLIST"

# ---------- convenience ranker ----------
def rank_stocks(bundles):
    """
    bundles = list of dicts, each:
      {'fund': stock_fundamentals_dict,
       'inc': income_df_q,
       'cf': cashflow_df_q,
       'bs': balance_df_q}
    Returns sorted list of (Company, Score, Label).
    """
    out = []
    for b in bundles:
        name = b['fund'].get('Company', 'Unknown')
        sc = buy_score(b['fund'], b['inc'], b['cf'], b['bs'])
        out.append((name, sc, score_label(sc)))
    return sorted(out, key=lambda x: x[1], reverse=True)

# --- end of simple_buy_score.py ------------------------------------------------


In [42]:
import math

# ---------- helpers ----------
def _to_float(x, default=0.0):
    if x is None:
        return default
    s = str(x).replace(',', '').strip()
    if s.endswith('%'):
        s = s[:-1]
    try:
        return float(s)
    except:
        return default

def _pos(x, lo, hi):
    """Higher is better. Linear 0..1 between [lo, hi]."""
    if x <= lo: return 0.0
    if x >= hi: return 1.0
    return (x - lo) / (hi - lo)

def _neg(x, lo, hi):
    """Lower is better. Linear 0..1 between [lo, hi]."""
    if x <= lo: return 1.0
    if x >= hi: return 0.0
    return 1.0 - (x - lo) / (hi - lo)

def _series_recent4(df, row_label):
    """Return list of 4 most-recent values for a given row (columns are most-recent first)."""
    if row_label not in df.index:
        return [0.0, 0.0, 0.0, 0.0]
    row = df.loc[row_label]
    vals = [_to_float(row.iloc[i], 0.0) for i in range(min(4, row.shape[0]))]
    while len(vals) < 4:
        vals.append(vals[-1] if vals else 0.0)
    return vals

def _weighted_recent4(values, weights=(1.0, 0.75, 0.50, 0.25)):
    return sum(values[i]*weights[i] for i in range(4)) / sum(weights)

# ---------- metric extraction from dfs ----------
def extract_from_quarterlies(income_df, cashflow_df, balance_df, fundamentals_dict):
    # Revenue TTM growth (current 4q vs previous 4q)
    rev_row = income_df.loc['Total Revenue'] if 'Total Revenue' in income_df.index else None
    if rev_row is not None and rev_row.shape[0] >= 8:
        rev_now  = sum(_to_float(rev_row.iloc[i])   for i in range(4))      # cols 0..3
        rev_prev = sum(_to_float(rev_row.iloc[i])   for i in range(4, 8))   # cols 4..7
        rev_ttm_growth = ( (rev_now - rev_prev)/rev_prev*100.0 ) if rev_prev > 0 else 0.0
    else:
        # Fallback to fundamentals 'Sales Y/Y TTM'
        rev_ttm_growth = _to_float(fundamentals_dict.get('Sales Y/Y TTM', 0.0))

    # FCF margin: weighted last 4 quarters (linear decay)
    fcf4 = _series_recent4(cashflow_df, 'Free Cash Flow')
    rev4 = _series_recent4(income_df, 'Total Revenue')
    fcf_w = _weighted_recent4(fcf4)
    rev_w = _weighted_recent4(rev4)
    fcf_margin = (fcf_w/rev_w*100.0) if rev_w > 0 else 0.0

    # Safety: Current Ratio from balance sheet (latest column 0)
    curr_ratio = _to_float(balance_df.loc['Current Ratio'].iloc[0]) if 'Current Ratio' in balance_df.index else _to_float(fundamentals_dict.get('Current Ratio', 1.0))

    # Debt/Equity: use fundamentals (simple and available)
    debt_eq = _to_float(fundamentals_dict.get('Debt/Eq', 0.0))

    return rev_ttm_growth, fcf_margin, debt_eq, curr_ratio

# ---------- final score ----------
def buy_score(stock_fundamentals, income_df, cashflow_df, balance_df):
    # From fundamentals (all are % except valuation)

    gm  = _to_float(stock_fundamentals.get('Gross Margin', 0))     # %
    om  = _to_float(stock_fundamentals.get('Oper. Margin', 0))     # %
    roe = _to_float(stock_fundamentals.get('ROE', 0))              # %
    fpe = _to_float(stock_fundamentals.get('Forward P/E', 0))      # number
    peg = _to_float(stock_fundamentals.get('PEG', 0))              # number

    # From dfs (or simple fallbacks)
    rev_g, fcf_margin, debt_eq, curr_ratio = extract_from_quarterlies(
        income_df, cashflow_df, balance_df, stock_fundamentals
    )

    # Subscores (0..1) — keep ranges simple and sane for a 2-year horizon
    growth = _pos(rev_g, 5, 40)  # 5%~40% revenue TTM growth
    profitability = (
        _pos(gm, 40, 70) +
        _pos(om, 15, 45) +
        _pos(roe, 10, 40) +
        _pos(fcf_margin, 5, 35)   # strong cash conversion
    ) / 4.0
    valuation = (
        _neg(fpe, 12, 40) +
        _neg(peg, 1.0, 3.0)
    ) / 2.0
    safety = (
        _neg(debt_eq, 0.0, 1.0) +
        _pos(curr_ratio, 1.0, 3.0)
    ) / 2.0

    score = (
        0.30*growth +
        0.30*profitability +
        0.25*valuation +
        0.15*safety
    ) * 100.0


    metric_vals = [gm, om, roe, fpe, peg, rev_g, fcf_margin, debt_eq, curr_ratio, growth, profitability, valuation, safety, score]
    return round(score, 1), metric_vals

def score_label(score):
    if score >= 75: return "BUY (High Conviction)"
    if score >= 60: return "ACCUMULATE / HOLD"
    return "AVOID / WATCHLIST"

# ---------- convenience ranker ----------
def rank_stocks(bundles):
    """
    bundles = list of dicts, each:
      {'fund': stock_fundamentals_dict,
       'inc': income_df_q,
       'cf': cashflow_df_q,
       'bs': balance_df_q}
    Returns sorted list of (Company, Score, Label).
    """
    out = []
    ticker_df = pd.DataFrame(columns=["Company", "Gross Margin", "Oper. Margin", "ROE", "Forward P/E", "PEG", "Revenue TTM Growth", "FCF Margin", "Debt/Equity", "Current Ratio", "Growth", "Profitability", "Valuation", "Safety", "Score"])
    for b in bundles:
        name = b['fund'].get('Company', 'Unknown')
        sc, metric_vals = buy_score(b['fund'], b['inc'], b['cf'], b['bs'])
        out.append((name, sc, score_label(sc)))

        ticker_df.loc[len(ticker_df)] = [name] + metric_vals
    return sorted(out, key=lambda x: x[1], reverse=True), ticker_df

# --- end of simple_buy_score.py ------------------------------------------------


In [43]:
bundles= []

for ticker in tickers:
    print(f'scraping: {ticker}')
    try:
        stock = finvizfinance(ticker)
        stock.ticker_charts(out_dir='asset')
    except Exception as e:
        print(f'error scraping {ticker}: {e}')
        continue

    stock_fundamentals = stock.ticker_fundament()
    stock_description = stock.ticker_description()
    outer_ratings_df = stock.ticker_outer_ratings()
    news_df = stock.ticker_news()
    inside_trader_df = stock.ticker_inside_trader()

    stock_charts = Statements()
    income_df_q= stock_charts.get_statements(ticker=ticker, statement='I', timeframe='Q')
    cashflow_df_q= stock_charts.get_statements(ticker=ticker, statement='C', timeframe='Q')
    balance_df_q= stock_charts.get_statements(ticker=ticker, statement='B', timeframe='Q')

    bundle= {
        'fund': stock_fundamentals,
        'inc': income_df_q,
        'cf': cashflow_df_q,
        'bs': balance_df_q
    }
    bundles.append(bundle)



scraping: EQIX
scraping: DLR
scraping: PWR
scraping: ETN
scraping: ABB
error scraping ABB: HTTP error for URL https://finviz.com/quote.ashx?t=ABB: 404 Client Error: Not Found for url: https://finviz.com/quote.ashx?t=ABB
scraping: VRT
scraping: TT
scraping: JCI
scraping: SMCI
scraping: HPE
scraping: DELL
scraping: ANET
scraping: AVGO
scraping: CSCO
scraping: MRVL
scraping: COHR
scraping: LITE
scraping: INFN
error scraping INFN: HTTP error for URL https://finviz.com/quote.ashx?t=INFN: 404 Client Error: Not Found for url: https://finviz.com/quote.ashx?t=INFN
scraping: MU
scraping: ON
scraping: STM
scraping: ADI
scraping: TXN
scraping: TSM
scraping: INTC
scraping: ASX
scraping: AMKR
scraping: PLAB
scraping: SNPS
scraping: CDNS
scraping: ANSS
error scraping ANSS: HTTP error for URL https://finviz.com/quote.ashx?t=ANSS: 404 Client Error: Not Found for url: https://finviz.com/quote.ashx?t=ANSS
scraping: STX
scraping: WDC
scraping: ASML
scraping: AMAT
scraping: KLAC
scraping: LRCX
scraping: NV

In [44]:
rankings, ticker_df = rank_stocks(bundles)

In [46]:
ticker_df.sort_values(by='Score', ascending=False)

,Company,Gross Margin,Oper. Margin,ROE,Forward P/E,PEG,Revenue TTM Growth,FCF Margin,Debt/Equity,Current Ratio,Growth,Profitability,Valuation,Safety,Score
34,NVIDIA Corp,69.85,58.09,109.42,28.75,1.46,71.553470,41.662778,0.11,4.21,1.000000,0.998750,0.585893,0.9450,88.784821
21,Taiwan Semiconductor Manufacturing ADR,58.06,49.54,34.89,23.69,1.00,39.728576,23.640788,0.19,2.69,0.992245,0.763257,0.791250,0.8275,84.858798
16,Micron Technology Inc,40.06,26.67,17.20,11.26,0.91,48.851101,5.218518,0.28,2.52,1.000000,0.159571,1.000000,0.7400,70.887130
7,Super Micro Computer Inc,11.06,5.70,17.90,14.53,1.49,47.040491,9.194142,0.76,5.25,1.000000,0.100785,0.832321,0.6200,63.131571
28,Seagate Technology Holdings Plc,35.18,21.05,0.00,16.94,1.29,38.864296,11.593135,0.00,1.38,0.967551,0.105359,0.839286,0.5950,62.094466
10,Arista Networks Inc,64.24,43.14,33.64,46.65,2.95,25.972682,47.542308,0.00,3.33,0.599219,0.883500,0.012500,1.0000,59.794085
13,Marvell Technology Inc,44.64,6.02,-0.75,25.07,0.00,37.053174,19.342354,0.38,1.88,0.915805,0.158186,0.766607,0.5300,59.334916
33,Lam Research Corp,49.31,33.00,62.26,27.20,2.30,25.658396,31.832989,0.44,2.21,0.590240,0.701192,0.403571,0.5825,57.569729
37,Adobe Inc,88.43,36.58,52.87,15.11,1.72,10.691433,38.769730,0.56,1.02,0.162612,0.929833,0.764464,0.2250,55.259979
32,KLA Corp,61.01,42.75,100.78,29.77,3.74,24.043917,31.298974,1.30,2.56,0.544112,0.875491,0.182679,0.3900,53.005065
